In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import pandas as pd
import os
import pickle
import numpy as np
from google.colab.patches import cv2_imshow

In [3]:
model_name = "Facenet512"
model_name = "ArcFace"
#option_movie_dir = ["like_me", "Calloused_Hands", "Memphis", "Liberty_Kid", "losing_ground"]
#option_movie_dir = "Calloused_Hands"
#option_movie_dir = "losing_ground"
#option_movie_dir = "Memphis"
#option_movie_dir = "Liberty_Kid"
option_movie_dir = "like_me"
option_character = "Burt"
#option_character = "Kiya"
#option_character = "sara"
#option_character = "Byrd"
#option_character = "Debbie"
#option_character = "willis"
#option_character = "Derrick"
Number_Img = 1
#lst_filname_face_query = ["Byrd_1.jpg","Byrd_2.jpg","Byrd_3.jpg","Byrd_4.jpg","Byrd_5.jpg","Byrd_6.jpg"]
#lst_filname_face_query = ["Debbie_1.jpg","Debbie_2.jpg","Debbie_3.jpg","Debbie_4.jpg","Debbie_5.jpg","Debbie_6.jpg"]
#lst_filname_face_query = ["willis_1.jpg","willis_2.jpg","willis_3.jpg","willis_4.jpg","willis_5.jpg","willis_6.jpg","willis_7.jpg"]
lst_filname_face_query = ["Burt1.jpg","Burt2.jpg","Burt3.jpg","Burt4.jpg","Burt5.jpg"]
#lst_filname_face_query = ["Derrick_1.jpg","Derrick_2.jpg","Derrick_3.jpg","Derrick_4.jpg","Derrick_5.jpg","Derrick_6.jpg","Derrick_7.jpg"]
#lst_filname_face_query = ["Kiya1.jpg","Kiya2.jpg","Kiya3.jpg","Kiya4.jpg","Kiya5.jpg","Kiya6.jpg"]
#lst_filname_face_query = ["sara_1.jpg","sara_2.jpg","sara_3.jpg","sara_4.jpg","sara_5.jpg"]
root_dataset = "/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022"

path_to_file_gt= f"{root_dataset}/ground_truth/{option_movie_dir}/{option_character}.xlsx"
path_to_folder_features =f"{root_dataset}/features_5fps/deepface/{model_name}/{option_movie_dir}"
path_to_file_index_fnames =f"/content/drive/MyDrive/KLTN2022_ThanhHa/source/faiss_index/{model_name}/{option_movie_dir}/{option_movie_dir}-fnames.pickle"


# Load file index fnames

In [4]:
with open(path_to_file_index_fnames, 'rb') as f:
  lst_face_name = pickle.load(f)
list_fname = lst_face_name

# Load ground truth

In [5]:
gt_path = pd.read_excel(path_to_file_gt)
df_ground_truth_full = gt_path["Full"]
#df_ground_truth_full
lst_groundtruth = list(df_ground_truth_full)

# Đọc feature face query

In [6]:
lst_name_face_query = []
arr_feature_query = []
for filname_face_query in lst_filname_face_query:
  name_face_query = filname_face_query.split('.')[0]
  path_to_file_feature_query  = f'{root_dataset}/features_query/deepface/{model_name}/{option_movie_dir}/{option_character}/{name_face_query}.pickle'
  path_to_file_face_query  = f'{root_dataset}/faces_query/{option_movie_dir}/{option_character}/{filname_face_query}'
  if os.path.exists(path_to_file_feature_query):
    filename_feature_query = open(path_to_file_feature_query, 'rb')
    feature_query = np.array(pickle.load(filename_feature_query))
    #feature_query = feature_query.reshape(1,-1)
    print(feature_query.shape)
    arr_feature_query.append(feature_query)
    lst_name_face_query.append(name_face_query)

(512,)
(512,)
(512,)
(512,)
(512,)


In [7]:
arr_feature_query = np.array(arr_feature_query)
arr_feature_query.shape

(5, 512)

In [8]:
lst_name_face_query

['Burt1', 'Burt2', 'Burt3', 'Burt4', 'Burt5']

# Đọc feature dataset

In [9]:
scenes_path = []
features_path = []

list_scenes = sorted(os.listdir(path_to_folder_features))
for scene in list_scenes:
  scene_path = os.path.join(path_to_folder_features,scene)
  scenes_path.append(scene_path)

for scene_path in scenes_path:
  list_features = sorted(os.listdir(scene_path))
  for feature in list_features:
    feature_path = os.path.join(scene_path,feature)
    features_path.append(feature_path)

In [10]:
features = []

for feature_path in features_path:
  filename_feature = open(feature_path, "rb")
  f = pickle.load(filename_feature)
  feature_data = f["Embedding"]
  features += feature_data
features = np.array(features)

# Thực hiện truy vấn

In [11]:
from scipy.spatial import distance

In [12]:
lst_cosine_dist = []

for i in range(arr_feature_query.shape[0]):
  cosine_dist = [distance.cosine(arr_feature_query[i].reshape(1,-1), f.reshape(1,-1)) for f in features]
  cosine_dist = np.array(cosine_dist)
  lst_cosine_dist.append(cosine_dist)

In [13]:
topk = len(lst_face_name)

indices = []
for x in range(len(lst_cosine_dist)):
  indi = lst_cosine_dist[x].argsort()[:topk]
  indices.append(indi)

In [14]:
lst_cosine_dist

[array([1.04322131, 1.04677524, 1.0369015 , ..., 0.92900938, 0.95178738,
        0.9394149 ]),
 array([0.88341813, 0.90804603, 0.89743768, ..., 0.87365579, 0.90053933,
        0.89575032]),
 array([0.43478612, 0.40817812, 0.37839151, ..., 0.0256233 , 0.04657607,
        0.04622992]),
 array([0.88272664, 0.87486328, 0.86621143, ..., 0.74604296, 0.75696924,
        0.7558395 ]),
 array([0.69310186, 0.66655085, 0.64769832, ..., 0.35394487, 0.38096952,
        0.37513064])]

In [15]:
indices

[array([13617, 13622, 13621, ..., 11639,  7938, 11640]),
 array([13562, 13568, 13571, ...,   176,  8567,  8573]),
 array([13816, 13578, 13877, ..., 15570, 13173, 13172]),
 array([11972, 11971, 11976, ..., 11645, 11647, 11646]),
 array([12165, 12162, 12167, ..., 16432, 13276, 13281])]

In [16]:
lst_indices = lst_cosine_dist[2].argsort()[:topk]
lst_distance = lst_cosine_dist[2]

In [17]:
lst_distance

array([0.43478612, 0.40817812, 0.37839151, ..., 0.0256233 , 0.04657607,
       0.04622992])

In [18]:
lst_indices

array([13816, 13578, 13877, ..., 15570, 13173, 13172])

In [ ]:
path_to_face = '/content/drive/MyDrive/KLTN2022_ThanhHa/dataset/TRECVID_MSUM_2022/faces_5fps/'

for ii in lst_indices:
  fname = list_fname[ii]
  arr_tmp = fname.split('-')
  path_fname = os.path.join(path_to_face, arr_tmp[0],f'{arr_tmp[0]}-{arr_tmp[1]}', f'{arr_tmp[0]}-{arr_tmp[1]}-{arr_tmp[2]}', f'{arr_tmp[0]}-{arr_tmp[1]}-{arr_tmp[2]}-{arr_tmp[3]}',fname)
  #print(path_fname)
  cv2_imshow(cv2.imread(path_fname))

# Chuyển hóa thành ma trận và tạo danh sách kết quả

In [19]:
MAX_SCENES = 100
MAX_SHOTS = 200

In [20]:
def Refine_Result(lst_distance, lst_indices, list_fname):
  matrix_distance_min = -np.ones((MAX_SCENES,MAX_SHOTS))
  
  for dist,idx in zip(lst_distance,lst_indices):
    predicted_face = list_fname[idx]
    #print(predicted_face)
    ind = predicted_face.split('-')
    scene_id = int(ind[1])
    shot_id = int(ind[2].split('_')[1])
    #print(f"scene_id={scene_id}, shot_id={shot_id}")    

    distance_min = matrix_distance_min[scene_id, shot_id]
    if distance_min==-1:
      distance_min = dist
    else:
      distance_min = min(dist,distance_min)

    matrix_distance_min[scene_id, shot_id] = distance_min
  return matrix_distance_min

In [21]:
def Create_List_Results(matrix_distance_min,option_movie_dir):
  lst_results = []
  idx_distance_min = np.argwhere(matrix_distance_min > -1)
  arr_distance = []
  for idx in idx_distance_min:
    arr_distance.append(matrix_distance_min[idx[0], idx[1]])
  arr_distance = np.array(arr_distance)
  idx_sorted = arr_distance.argsort()
  
  for jdx in idx_sorted:
    result = f'{option_movie_dir}-{idx_distance_min[jdx][0]}-shot_{idx_distance_min[jdx][1]}'
    lst_results.append(result)
  return lst_results

**Đối với một face query**

In [22]:
matrix_distance_min =  Refine_Result(lst_distance,lst_indices,list_fname)
lst_results = Create_List_Results(matrix_distance_min,option_movie_dir)

In [23]:
lst_results

['like_me-3-shot_48',
 'like_me-3-shot_56',
 'like_me-9-shot_11',
 'like_me-19-shot_14',
 'like_me-4-shot_2',
 'like_me-5-shot_4',
 'like_me-22-shot_2',
 'like_me-13-shot_20',
 'like_me-23-shot_2',
 'like_me-7-shot_25',
 'like_me-22-shot_14',
 'like_me-7-shot_2',
 'like_me-26-shot_1',
 'like_me-20-shot_73',
 'like_me-22-shot_0',
 'like_me-22-shot_4',
 'like_me-15-shot_21',
 'like_me-7-shot_48',
 'like_me-7-shot_30',
 'like_me-7-shot_24',
 'like_me-15-shot_2',
 'like_me-9-shot_18',
 'like_me-7-shot_23',
 'like_me-20-shot_13',
 'like_me-21-shot_0',
 'like_me-27-shot_51',
 'like_me-10-shot_3',
 'like_me-14-shot_2',
 'like_me-22-shot_7',
 'like_me-11-shot_18',
 'like_me-24-shot_0',
 'like_me-23-shot_3',
 'like_me-20-shot_9',
 'like_me-7-shot_38',
 'like_me-6-shot_3',
 'like_me-18-shot_7',
 'like_me-7-shot_16',
 'like_me-3-shot_91',
 'like_me-7-shot_45',
 'like_me-4-shot_34',
 'like_me-20-shot_61',
 'like_me-7-shot_47',
 'like_me-28-shot_30',
 'like_me-8-shot_1',
 'like_me-19-shot_9',
 'lik

**Đối với nhiều face query**

In [29]:
lst_matrix_distance_min=[]
lst_lst_results = []
for i in range(len(indices)):
  matrix_distance_min =  Refine_Result(lst_cosine_dist[i],indices[i],list_fname)
  
  lst_results = Create_List_Results(matrix_distance_min,option_movie_dir)
  
  lst_matrix_distance_min.append(matrix_distance_min)  
  lst_lst_results.append(lst_results)

In [30]:
lst_lst_results

[['like_me-20-shot_18',
  'like_me-18-shot_6',
  'like_me-26-shot_10',
  'like_me-1-shot_1',
  'like_me-2-shot_1',
  'like_me-10-shot_2',
  'like_me-18-shot_3',
  'like_me-5-shot_20',
  'like_me-17-shot_13',
  'like_me-10-shot_3',
  'like_me-22-shot_7',
  'like_me-19-shot_13',
  'like_me-20-shot_20',
  'like_me-11-shot_13',
  'like_me-1-shot_5',
  'like_me-17-shot_16',
  'like_me-13-shot_24',
  'like_me-11-shot_16',
  'like_me-15-shot_21',
  'like_me-11-shot_8',
  'like_me-19-shot_3',
  'like_me-2-shot_34',
  'like_me-2-shot_9',
  'like_me-3-shot_61',
  'like_me-18-shot_9',
  'like_me-7-shot_48',
  'like_me-15-shot_7',
  'like_me-22-shot_1',
  'like_me-2-shot_48',
  'like_me-28-shot_29',
  'like_me-7-shot_14',
  'like_me-11-shot_10',
  'like_me-11-shot_19',
  'like_me-7-shot_6',
  'like_me-21-shot_0',
  'like_me-17-shot_27',
  'like_me-26-shot_0',
  'like_me-4-shot_24',
  'like_me-11-shot_4',
  'like_me-20-shot_38',
  'like_me-20-shot_26',
  'like_me-7-shot_30',
  'like_me-7-shot_32',


**Tạo danh sách kết quả cho tất cả face query**

In [31]:
matrix_matrix_result = lst_matrix_distance_min[0]

for j in range(1, len(lst_matrix_distance_min)):
  for r1 in range(MAX_SCENES):
    for c1 in range(MAX_SHOTS):
      if (matrix_matrix_result[r1][c1] == -1) and (lst_matrix_distance_min[j][r1][c1] == -1):
        matrix_matrix_result[r1][c1] = -1
      if (matrix_matrix_result[r1][c1] != -1) and (lst_matrix_distance_min[j][r1][c1] != -1):
        matrix_matrix_result[r1][c1] = min(lst_matrix_distance_min[j][r1][c1],matrix_matrix_result[r1][c1])
      if (matrix_matrix_result[r1][c1] == -1) and (lst_matrix_distance_min[j][r1][c1] != -1):
        matrix_matrix_result[r1][c1] = lst_matrix_distance_min[j][r1][c1]
      if (matrix_matrix_result[r1][c1] != -1) and (lst_matrix_distance_min[j][r1][c1] == -1):
        matrix_matrix_result[r1][c1] = matrix_matrix_result[r1][c1]

In [32]:
lst_lst_lst_results = Create_List_Results(matrix_matrix_result,option_movie_dir)

In [33]:
lst_lst_lst_results

['like_me-3-shot_48',
 'like_me-3-shot_56',
 'like_me-9-shot_11',
 'like_me-19-shot_14',
 'like_me-4-shot_2',
 'like_me-5-shot_4',
 'like_me-22-shot_2',
 'like_me-13-shot_20',
 'like_me-23-shot_2',
 'like_me-7-shot_25',
 'like_me-22-shot_14',
 'like_me-7-shot_2',
 'like_me-26-shot_1',
 'like_me-20-shot_73',
 'like_me-22-shot_0',
 'like_me-22-shot_4',
 'like_me-15-shot_21',
 'like_me-7-shot_48',
 'like_me-7-shot_30',
 'like_me-7-shot_24',
 'like_me-15-shot_2',
 'like_me-9-shot_18',
 'like_me-7-shot_23',
 'like_me-20-shot_13',
 'like_me-21-shot_0',
 'like_me-27-shot_51',
 'like_me-10-shot_3',
 'like_me-14-shot_2',
 'like_me-22-shot_7',
 'like_me-11-shot_18',
 'like_me-24-shot_0',
 'like_me-23-shot_3',
 'like_me-20-shot_9',
 'like_me-7-shot_38',
 'like_me-6-shot_3',
 'like_me-18-shot_7',
 'like_me-7-shot_16',
 'like_me-3-shot_91',
 'like_me-7-shot_45',
 'like_me-4-shot_34',
 'like_me-20-shot_61',
 'like_me-7-shot_47',
 'like_me-28-shot_30',
 'like_me-8-shot_1',
 'like_me-19-shot_9',
 'lik

# Đánh giá kết quả

In [24]:
# Evaluation Metrics For Information Retrieval
def EvaluationMetrics(lst_predict, lst_groundtruth):
    num_gt = len(lst_groundtruth)
    num_correct = 0
    pos_predict = 0
    arr_Precision = []
    arr_Recall = []
    arr_F1 = []
    arr_AP = []
    myAP = 0
    arr_Reciprocal_Rank = []
    
    First_Reciprocal_Rank = -1
    for predict in lst_predict:
        pos_predict = pos_predict +1
        #print(f"predict = {predict}")
        if predict in lst_groundtruth:
            #print(f"predict in lst_groundtruth = {predict}")
            num_correct += 1
            myAP = myAP + num_correct/pos_predict
            Reciprocal_Rank = 1.0/pos_predict

            arr_Reciprocal_Rank.append(Reciprocal_Rank )
            if First_Reciprocal_Rank == -1:
                First_Reciprocal_Rank = Reciprocal_Rank
        else:
            arr_Reciprocal_Rank.append(0)
        
        Precision_k = num_correct/pos_predict
        arr_Precision.append(Precision_k)

        Recall_k = num_correct/num_gt
        arr_Recall.append(Recall_k)
        if Precision_k + Recall_k >0:
          F1_k = 2*Precision_k*Recall_k/(Precision_k+Recall_k)
        else:
          F1_k = 0

        arr_F1.append(F1_k)
        if num_correct>0:
          arr_AP.append(myAP/num_correct)
        else:
          arr_AP.append(0)

    return First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP

**Đối với một face query**

In [25]:
lst_predict = lst_results

In [26]:
First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP = EvaluationMetrics(lst_predict, lst_groundtruth)

In [27]:
print("AP_1=%5.2f" % arr_AP[0], 
      "AP_5=%5.2f" % arr_AP[4],
      "AP_10=%5.2f" % arr_AP[9], 
      "AP_50=%5.2f"% arr_AP[49], 
      "AP_100=%5.2f"% arr_AP[99])

AP_1= 0.00 AP_5= 0.20 AP_10= 0.20 AP_50= 0.12 AP_100= 0.09


**Đối với nhiều face query**

In [34]:
lst_lst_predict = lst_lst_lst_results

In [35]:
First_Reciprocal_Rank, arr_Reciprocal_Rank, arr_Precision, arr_Recall, arr_F1, arr_AP = EvaluationMetrics(lst_lst_predict, lst_groundtruth)

In [36]:
print("AP_1=%5.2f" % arr_AP[0], 
      "AP_5=%5.2f" % arr_AP[4],
      "AP_10=%5.2f" % arr_AP[9], 
      "AP_50=%5.2f"% arr_AP[49], 
      "AP_100=%5.2f"% arr_AP[99])

AP_1= 0.00 AP_5= 0.20 AP_10= 0.20 AP_50= 0.12 AP_100= 0.09
